# Deep Learning models using Keras

Keras is a user-friendly neural network library written in Python. We will build a regression model to predict an employee's wage per hour.

Note: The datasets we will be using are relatively clean, so we will not perform any data preprocessing in order to get our data ready for modeling. Datasets that you will use in future projects may not be so clean - for example, they may have missing values - so you may need to use data preprocessing techniques to alter your datasets to get more accurate results.

# Regression Model using Keras

For our regression deep learning model, the first step is to read in the data we will use as input. For this example, we are using the 'hourly wages' dataset. To start, we will use Pandas to read in the data. I will not go into detail on Pandas, but it is a library you should become familiar with if you're looking to dive further into data science and machine learning.

'df' stands for dataframe. Pandas read in the CSV file as a dataframe. The 'head()' function will show the first 5 rows of the dataframe so you can check that the data has been read in properly and can take an initial look at how the data is structured.

In [19]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [4]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/IntroToDNNwKeras.git cloned-repo
%cd cloned-repo

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo


In [3]:
#read in training data
wage_df = pd.read_csv('/content/cloned-repo/hourly_wage')

#view data structure
wage_df.head()
print(wage_df.shape)


(534, 10)


Next, we need to split up our dataset into inputs (train_X) and our target (train_y). Our input will be every column except 'wage_per_hour' because 'wage_per_hour' is what we will be attempting to predict. Therefore, 'wage_per_hour' will be our target.

We will use the pandas 'drop' function to drop the column 'wage_per_hour' from our dataframe and store it in the variable 'train_X'. This will be our input.

In [4]:
#create a dataframe with all training data except the target column
wage_data = wage_df.drop(columns=['wage_per_hour'])

#check that the target variable has been removed
wage_data.head()

,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
0,0,8,21,35,1,1,0,1,0
1,0,9,42,57,1,1,0,1,0
2,0,12,1,19,0,0,0,1,0
3,0,12,4,22,0,0,0,0,0
4,0,12,17,35,0,1,0,0,0


In [5]:
#create a label column with only the target column
wage_y = wage_df[['wage_per_hour']]

#view dataframe
wage_y.head()

,wage_per_hour
0,5.10
1,4.95
2,6.67
3,4.00
4,7.50


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    wage_data, wage_y, test_size=0.20)

The model type that we will be using is Sequential. Sequential is the easiest way to build a model in Keras. It allows you to build a model layer by layer. Each layer has weights that correspond to the layer the follows it.

We use the 'add()' function to add layers to our model. We will add two layers and an output layer.

'Dense' is the layer type. Dense is a standard layer type that works for most cases. In a dense layer, all nodes in the previous layer connect to the nodes in the current layer.

We have 10 nodes in each of our input layers. This number can also be in the hundreds or thousands. Increasing the number of nodes in each layer increases model capacity. I will go into further detail about the effects of increasing model capacity shortly.

'Activation' is the activation function for the layer. An activation function allows models to take into account nonlinear relationships. For example, if you are predicting diabetes in patients, going from age 10 to 11 is different than going from age 60â??61.

The activation function we will be using is ReLU or Rectified Linear Activation. Although it is two linear pieces, it has been proven to work well in neural networks.

The first layer needs an input shape. The input shape specifies the number of rows and columns in the input. The number of columns in our input is stored in 'n_cols'. There is nothing after the comma which indicates that there can be any amount of rows.

The last layer is the output layer. It only has one node, which is for our prediction.

Next, we need to compile our model. Compiling the model takes two parameters: optimizer and loss.

The optimizer controls the learning rate. We will be using 'Adam' as our optimizer. Adam is generally a good optimizer to use for many cases. The Adam optimizer adjusts the learning rate throughout training.

The learning rate determines how fast the optimal weights for the model are calculated. A smaller learning rate may lead to more accurate weights (up to a certain point), but the time it takes to compute the weights will be longer.

For our loss function, we will use 'mean_squared_error'. It is calculated by taking the average squared difference between the predicted and actual values. It is a popular loss function for regression problems. The closer to 0 this is, the better the model performed.



In [7]:
#create model
model = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]

#add model layers
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

#compile model using mse as a measure of model performance
model.compile(optimizer='adam', loss='mean_squared_error', metrics=["mean_squared_error"])

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

Now we will train our model. To train, we will use the 'fit()' function on our model with the following five parameters: training data (train_X), target data (train_y), validation split, the number of epochs and callbacks.

The validation split will randomly split the data into use for training and testing. During training, we will be able to see the validation loss, which gives the mean squared error of our model on the validation set. We will set the validation split at 0.2, which means that 20% of the training data we provide in the model will be set aside for testing model performance.

The number of epochs is the number of times the model will cycle through the data. The more epochs we run, the more the model will improve, up to a certain point. After that point, the model will stop improving during each epoch. In addition, the more epochs, the longer the model will take to run. To monitor this, we will use 'early stopping'.

Early stopping will stop the model from training before the number of epochs is reached if the model stops improving. We will set our early stopping monitor to 3. This means that after 3 epochs in a row in which the model doesn't improve, training will stop. Sometimes, the validation loss can stop improving then improve in the next epoch, but after 3 epochs in which the validation loss doesn't improve, it usually won't improve again.

In [8]:
#train model
model.fit(X_train, y_train, validation_split=0.2, epochs=30, 
          callbacks=[early_stopping_monitor])

Epoch 1/30
11/11 [==============================] - 2s 51ms/step - loss: 403.3515 - mean_squared_error: 403.3515 - val_loss: 448.7564 - val_mean_squared_error: 448.7564
Epoch 2/30
11/11 [==============================] - 0s 19ms/step - loss: 346.8295 - mean_squared_error: 346.8295 - val_loss: 389.1834 - val_mean_squared_error: 389.1834
Epoch 3/30
11/11 [==============================] - 0s 33ms/step - loss: 300.1179 - mean_squared_error: 300.1179 - val_loss: 339.0874 - val_mean_squared_error: 339.0874
Epoch 4/30
11/11 [==============================] - 0s 16ms/step - loss: 260.1958 - mean_squared_error: 260.1958 - val_loss: 297.5929 - val_mean_squared_error: 297.5929
Epoch 5/30
11/11 [==============================] - 0s 18ms/step - loss: 227.3242 - mean_squared_error: 227.3242 - val_loss: 261.3395 - val_mean_squared_error: 261.3395
Epoch 6/30
11/11 [==============================] - 0s 16ms/step - loss: 196.5257 - mean_squared_error: 196.5257 - val_loss: 224.2433 - val_mean_squared_er

Making predictions on new data

If you want to use this model to make predictions on new data, we would use the 'predict()' function, passing in our new data.
The output would be 'wage_per_hour' predictions.

In [9]:
wage_pred = model.predict(X_test)

4/4 [==============================] - 0s 4ms/step


In [10]:
y_test_wages=y_test[["wage_per_hour"]].values
#y_test_wages

In [11]:
#print("prediction", "\tground truth")
#for i in range(len(y_test_wages)):
#  print(wage_pred[i],"\t", y_test_wages[i])

In [18]:
import numpy as np
from sklearn.metrics import mean_squared_error
rms_dnn_1=np.sqrt(mean_squared_error(y_test, wage_pred))
rms_dnn_1

NameError: ignored

As you increase the number of nodes and layers in a model, the model capacity increases. Increasing model capacity can lead to a more accurate model, up to a certain point, at which the model will stop improving. Generally, the more training data you provide, the larger the model should be. We are only using a tiny amount of data, so our model is pretty small. The larger the model, the more computational capacity it requires and it will take longer to train.

Let's create a new model using the same training data as our previous model. This time, we will add a layer and increase the nodes in each layer to 200. We will train the model to see if increasing the model capacity will improve our validation score.

In [15]:
#training a new model on the same data to show the effect of increasing model capacity
#create model
model_mc = Sequential()

#add model layers
model_mc.add(Dense(200, activation='relu', input_shape=(n_cols,)))
model_mc.add(Dense(200, activation='relu'))
model_mc.add(Dense(200, activation='relu'))
model_mc.add(Dense(1))

#compile model using mse as a measure of model performance
model_mc.compile(optimizer='adam', loss='mean_squared_error', metrics=["mean_squared_error"])

In [16]:
#train model
model_mc.fit(X_train, y_train, validation_split=0.2, 
             epochs=30, callbacks=[early_stopping_monitor])

Epoch 1/30
11/11 [==============================] - 1s 22ms/step - loss: 33.5541 - mean_squared_error: 33.5541 - val_loss: 36.4975 - val_mean_squared_error: 36.4975
Epoch 2/30
11/11 [==============================] - 0s 8ms/step - loss: 19.5303 - mean_squared_error: 19.5303 - val_loss: 36.8342 - val_mean_squared_error: 36.8342
Epoch 3/30
11/11 [==============================] - 0s 6ms/step - loss: 18.9747 - mean_squared_error: 18.9747 - val_loss: 38.4326 - val_mean_squared_error: 38.4326
Epoch 4/30
11/11 [==============================] - 0s 6ms/step - loss: 19.0343 - mean_squared_error: 19.0343 - val_loss: 40.1005 - val_mean_squared_error: 40.1005


In [17]:
wage_pred_mc = model_mc.predict(X_test)

4/4 [==============================] - 0s 3ms/step


In [18]:
rms_dnn=np.sqrt(mean_squared_error(y_test, wage_pred_mc))
rms_dnn

4.663942543811805

In [19]:
#print("prediction", "\tground truth")
#for i in range(len(y_test_wages)):
#  print(wage_pred_mc[i],"\t", y_test_wages[i])


In [20]:
from sklearn.linear_model import LinearRegression
model_sklearn=LinearRegression().fit(X_train, y_train)
pred_sklearn=model_sklearn.predict(X_test)

In [21]:
model_sklearn.score(X_test, y_test)

0.2513139670696958

In [22]:
import numpy as np
from sklearn.metrics import mean_squared_error
rms_sklearn=np.sqrt(mean_squared_error(y_test, pred_sklearn))
rms_sklearn

4.145285734352979

In [24]:
print("sklearn model:", rms_sklearn) 
print("dnn model:",rms_dnn)
print( "first dnn model:", rms_dnn_1)

sklearn model: 4.145285734352979
dnn model: 4.663942543811805
first dnn model: 5.265263723717107




---



---



# Classification Model using Keras

Now let's move on to building our model for classification. Since many steps will be a repeat from the previous model, I will only go over new concepts.

For this next model, we are going to predict if patients have diabetes or not.


   Given is the attribute name, attribute type, the measurement unit and a
   brief description.  The number of rings is the value to predict: either
   as a continuous value or as a classification problem.

	Name		Data Type	Meas.	Description
	----		---------	-----	-----------
	Sex		nominal			M, F, and I (infant)
	Length		continuous	mm	Longest shell measurement
	Diameter	continuous	mm	perpendicular to length
	Height		continuous	mm	with meat in shell
	Whole weight	continuous	grams	whole abalone
	Shucked weight	continuous	grams	weight of meat
	Viscera weight	continuous	grams	gut weight (after bleeding)
	Shell weight	continuous	grams	after being dried
	Rings		integer			+1.5 gives the age in years


Predicting the age of abalone from physical measurements. The age of abalone is determined by cutting the shell through the cone, staining it, and counting the number of rings through a microscope -- a boring and time-consuming task. Other measurements, which are easier to obtain, are used to predict the age. Further information, such as weather patterns and location (hence food availability) may be required to solve the problem.

In [116]:
#read in training data
train_df_2 = pd.read_csv('/content/abalone.data')

#view data structure
train_df_2.columns

Index(['M', '0.455', '0.365', '0.095', '0.514', '0.2245', '0.101', '0.15',
       '15'],
      dtype='object')

In [124]:
#train_df_2['M'] = train_df_2.M.astype('category')
#train_df_2["M"]
train_df_2=train_df_2.drop(columns=['M'])

In [125]:
cols=train_df_2.columns
train_df_2 = train_df_2.astype({'0.455':'float', '0.365':'float', '0.095':'float', '0.514':'float', '0.2245':'float', '0.101':'float', '0.15':'float'})

In [126]:
#create a dataframe with all training data except the target column
train_X_2 = train_df_2.drop(columns=['15'])
train_y_2 = train_df_2[["15"]]
#check that the target variable has been removed
train_X_2.head()

,0.455,0.365,0.095,0.514,0.2245,0.101,0.15
0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
3,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055
4,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120


In [127]:
train_y_2

,15
0,7
1,9
2,10
3,7
4,8
...,...
4171,11
4172,10
4173,9
4174,10


When separating the target column, we need to call the 'to_categorical()' function so that column will be 'one-hot encoded'. Currently, a patient with no diabetes is represented with a 0 in the diabetes column and a patient with diabetes is represented with a 1. 

With one-hot encoding, the integer will be removed and a binary variable is inputted for each category. In our case, we have two categories: no diabetes and diabetes.

A patient with no diabetes will be represented by [1 0] and a patient with diabetes will be represented by [0 1].

In [128]:
#one-hot encode target column
train_y_2 = to_categorical(train_y_2,30)

#vcheck that target column has been converted
train_y_2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

The last layer of our model has 2 nodes - one for each option: the patient has diabetes or they don't.

The activation is 'softmax'. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has a higher probability.

We will use 'categorical_crossentropy' for our loss function. This is the most common choice for classification. A lower score indicates that the model is performing better.

To make things even easier to interpret, we will use the 'accuracy' metric to see the accuracy score on the validation set at the end of each epoch.

In [129]:

print(train_X_2.shape)
outputs=train_y_2.shape
print(outputs)

(4176, 7)
(4176, 30)


In [133]:
#create model
model_2 = Sequential()
#get number of columns in training data
inputs=train_X_2.shape[1]
#add layers to model
model_2.add(Dense(250, activation='relu', input_shape=(inputs,)))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(30, activation='softmax'))
model_2.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 250)               2000      
                                                                 
 dense_47 (Dense)            (None, 250)               62750     
                                                                 
 dense_48 (Dense)            (None, 250)               62750     
                                                                 
 dense_49 (Dense)            (None, 30)                7530      
                                                                 
Total params: 135,030
Trainable params: 135,030
Non-trainable params: 0
_________________________________________________________________


In [134]:
#compile model using accuracy to measure model performance
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [135]:
import numpy as np
y = np.asarray(train_y_2).astype(np.int32)
print(y[1])

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [113]:
#train_X_2 = np.array([np.array(val) for val in train_X_2])

In [136]:
#train model
model_2.fit(train_X_2, train_y_2, epochs=30)

Epoch 1/30
131/131 [==============================] - 2s 5ms/step - loss: 2.4335 - accuracy: 0.2038
Epoch 2/30
131/131 [==============================] - 1s 5ms/step - loss: 2.1415 - accuracy: 0.2452
Epoch 3/30
131/131 [==============================] - 1s 5ms/step - loss: 2.0665 - accuracy: 0.2538
Epoch 4/30
131/131 [==============================] - 1s 5ms/step - loss: 2.0201 - accuracy: 0.2593
Epoch 5/30
131/131 [==============================] - 1s 5ms/step - loss: 1.9911 - accuracy: 0.2639
Epoch 6/30
131/131 [==============================] - 1s 5ms/step - loss: 1.9847 - accuracy: 0.2644
Epoch 7/30
131/131 [==============================] - 1s 5ms/step - loss: 1.9662 - accuracy: 0.2617
Epoch 8/30
131/131 [==============================] - 1s 5ms/step - loss: 1.9570 - accuracy: 0.2680
Epoch 9/30
131/131 [==============================] - 1s 5ms/step - loss: 1.9561 - accuracy: 0.2591
Epoch 10/30
131/131 [==============================] - 1s 8ms/step - loss: 1.9503 - accuracy: 0.2727